A notebook for organizing work for the WCAI-Evite datathon

Team members: 
Levani Zandarashvili  
Will Hu  
Leon Kim  
Mike Tao  
Rachael Creager  

Canvas link: https://canvas.upenn.edu/courses/1443789  
Student data: https://www.dropbox.com/home/Student%20Data

In [1]:
import warnings
warnings.filterwarnings('ignore')
import psycopg2
import pandas as pd
from datetime import datetime

In [2]:
try:
    conn =psycopg2.connect("dbname=postgres user=postgres password=password host=35.236.196.171")
    cur = conn.cursor()
    cur.execute("""SELECT table_name FROM information_schema.tables WHERE table_schema = 'public'""")
    print("tables available:")
    for table in cur.fetchall():
        print(table)
        
except (Exception, psycopg2.Error) as error :
    print("could not connect to postgres database")

tables available:
('host_keys',)
('event_keys',)
('events',)
('user_keys',)
('guest_hosts_tmp',)
('guest_hosts',)


In [12]:
cur.execute("""SELECT * from events limit 10;""")
evt_colnames = [desc[0] for desc in cur.description]

df = pd.DataFrame(cur.fetchall(), columns=evt_colnames)
df.drop('template_name', axis=1, inplace=True)
print(list(df.columns.values))

dates = df['event_date']
#print(dates)

times = [date.to_pydatetime(date).hour for date in dates]
day_part = ['Morning' if (time >= 0 and time <= 11)
            else 'Afternoon' if (time >= 12 and time <= 17)
            else 'Evening' if (time >= 8 and time <= 23)
            else 'Unknown'
            for time in times]
#print(day_part)

months = [date.to_pydatetime(date).month for date in dates]
seasons = [(month%12 + 3)//3 for month in months]
seasons_text = ['Winter' if s == 1 
                else 'Spring' if s == 2 
                else 'Summer' if s == 3 
                else 'Fall' if s == 4 
                else 'Unknown' for s in seasons]
#print(seasons_text)

created_dates = df['created_date']
created_dates = [create_date.to_pydatetime(create_date) for create_date in created_dates]
event_dates = df['event_date']
event_dates = [event_date.to_pydatetime(event_date) for event_date in event_dates]
create_event_diff = [event - created for event, created in zip(event_dates, created_dates)]
create_event_diff = [td.days for td in create_event_diff]
#print(create_event_diff)

mail_dates = df['sent_on']
mail_dates = [mail_date.to_pydatetime(mail_date) for mail_date in mail_dates]
mail_event_diff = [event - mail for event, mail in zip(event_dates, mail_dates)]
mail_event_diff = [td.days for td in mail_event_diff]
#print(mail_event_diff)

create_mail_diff = [created - mail for created, mail in zip(created_dates, mail_dates)]
create_mail_diff = [td.days for td in create_mail_diff]

['event_id', 'host_id', 'event_type', 'event_category', 'correspondence', 'event_category_group', 'seasonal', 'event_date', 'sent_on', 'created_date', 'location', 'zip', 'premium_vs_free', 'guests', 'rsvp_yes', 'rsvp_no', 'rsvp_maybe', 'rsvp_undecided']
